In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Load the dataset
file_path = "everyday_2024_w_metro_station.csv"
df = pd.read_csv(file_path)

# Display first few rows to inspect data
print(df.head())

FileNotFoundError: [Errno 2] No such file or directory: 'everyday_2024_w_metro_station.csv'

In [ ]:
# Define time period columns
traffic_columns = [
    "Entries (AM Peak (Open-9:30am))",
    "Entries (Midday (9:30am-3pm))",
    "Entries (PM Peak (3pm-7pm))",
    "Entries (Evening (7pm-12am))",
    "Entries (Late Night (12am-Close))"
]

# Convert entry columns to numeric (fixes errors where numbers are stored as text)
for col in traffic_columns:
    df[col] = pd.to_numeric(df[col], errors="coerce")
    
# Display summary statistics to check conversion
df[traffic_columns].describe()

,Entries (AM Peak (Open-9:30am)),Entries (Midday (9:30am-3pm)),Entries (PM Peak (3pm-7pm)),Entries (Evening (7pm-12am)),Entries (Late Night (12am-Close))
count,35752.000000,35752.000000,35752.000000,35752.000000,35752.000000
mean,871.777691,925.354665,1268.467666,499.832960,18.092722
std,840.445564,740.037515,1592.280650,806.400358,44.060204
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,218.000000,408.000000,327.000000,119.000000,3.000000
50%,646.000000,703.000000,638.000000,229.000000,8.000000
75%,1298.000000,1215.000000,1505.000000,560.000000,16.000000
max,6596.000000,19233.000000,19275.000000,16405.000000,2122.000000


In [2]:
# Compute total foot traffic per station per day
df["Total_Foot_Traffic"] = df[traffic_columns].sum(axis=1)

# Drop stations where Total_Foot_Traffic is 0 (closed stations)
df = df[df["Total_Foot_Traffic"] > 0]

# Display first few rows after filtering
df.head()

NameError: name 'df' is not defined

In [83]:
# Reshape the dataset so each row represents a specific station, date, and time period
df_melted = df.melt(
    id_vars=["Station Name", "Date"], 
    value_vars=traffic_columns, 
    var_name="Time Period", 
    value_name="Foot_Traffic"
)

# Display first few rows to confirm restructuring
df_melted.head()


,Station Name,Date,Time Period,Foot_Traffic
0,Addison Road,2024-01-01,Entries (AM Peak (Open-9:30am)),100
1,Addison Road,2024-01-02,Entries (AM Peak (Open-9:30am)),642
2,Addison Road,2024-01-03,Entries (AM Peak (Open-9:30am)),864
3,Addison Road,2024-01-04,Entries (AM Peak (Open-9:30am)),858
4,Addison Road,2024-01-05,Entries (AM Peak (Open-9:30am)),722


In [84]:
# Compute rolling average per station and time period (to smooth fluctuations)
df_melted["Station_Avg"] = (
    df_melted.groupby(["Station Name", "Time Period"])["Foot_Traffic"]
    .transform(lambda x: x.rolling(window=7, min_periods=1).mean())
)
# Switched from .rolling

# Display a sample of station averages
df_melted[["Station Name", "Time Period", "Station_Avg"]].drop_duplicates().head(20)

,Station Name,Time Period,Station_Avg
0,Addison Road,Entries (AM Peak (Open-9:30am)),100.000000
1,Addison Road,Entries (AM Peak (Open-9:30am)),371.000000
2,Addison Road,Entries (AM Peak (Open-9:30am)),535.333333
3,Addison Road,Entries (AM Peak (Open-9:30am)),616.000000
4,Addison Road,Entries (AM Peak (Open-9:30am)),637.200000
5,Addison Road,Entries (AM Peak (Open-9:30am)),553.000000
6,Addison Road,Entries (AM Peak (Open-9:30am)),487.714286
7,Addison Road,Entries (AM Peak (Open-9:30am)),590.857143
8,Addison Road,Entries (AM Peak (Open-9:30am)),627.428571
9,Addison Road,Entries (AM Peak (Open-9:30am)),632.428571


In [86]:
# Ensure no division errors
df_melted["Station_Avg"] = df_melted["Station_Avg"].replace(0, np.nan)  # Avoid division by zero

# Compute Dynamic Pricing Percentage with a safeguard against excessive negative values
df_melted["Dynamic_Price_Percentage"] = ((df_melted["Foot_Traffic"] - df_melted["Station_Avg"]) / df_melted["Station_Avg"]) * 100

# Ensure minimum value is -100% (foot traffic should never be less than 0)
df_melted["Dynamic_Price_Percentage"] = df_melted["Dynamic_Price_Percentage"].clip(lower=-100)

# Use percentile-based scaling instead of a hard cap
upper_limit = df_melted["Dynamic_Price_Percentage"].quantile(0.99)  # 99th percentile as cap
lower_limit = df_melted["Dynamic_Price_Percentage"].quantile(0.01)  # 1st percentile as min

# Clip based on real data distribution
df_melted["Dynamic_Price_Percentage"] = df_melted["Dynamic_Price_Percentage"].clip(lower=lower_limit, upper=upper_limit)

# Display first few rows to check calculation
df_melted[["Station Name", "Date", "Time Period", "Foot_Traffic", "Station_Avg", "Dynamic_Price_Percentage"]].head(20)


,Station Name,Date,Time Period,Foot_Traffic,Station_Avg,Dynamic_Price_Percentage
0,Addison Road,2024-01-01,Entries (AM Peak (Open-9:30am)),100,100.000000,0.000000
1,Addison Road,2024-01-02,Entries (AM Peak (Open-9:30am)),642,371.000000,73.045822
2,Addison Road,2024-01-03,Entries (AM Peak (Open-9:30am)),864,535.333333,61.394770
3,Addison Road,2024-01-04,Entries (AM Peak (Open-9:30am)),858,616.000000,39.285714
4,Addison Road,2024-01-05,Entries (AM Peak (Open-9:30am)),722,637.200000,13.308223
5,Addison Road,2024-01-06,Entries (AM Peak (Open-9:30am)),132,553.000000,-76.130199
6,Addison Road,2024-01-07,Entries (AM Peak (Open-9:30am)),96,487.714286,-80.316344
7,Addison Road,2024-01-08,Entries (AM Peak (Open-9:30am)),822,590.857143,39.119923
8,Addison Road,2024-01-09,Entries (AM Peak (Open-9:30am)),898,627.428571,43.123862
9,Addison Road,2024-01-10,Entries (AM Peak (Open-9:30am)),899,632.428571,42.150440


In [89]:
# 🔹 Sort by foot traffic (most crowded times appear first)
df_sorted = df_melted.sort_values(by="Foot_Traffic", ascending=False)

# Display top 20 busiest station-time slots with adjusted pricing
df_sorted[["Station Name", "Date", "Time Period", "Foot_Traffic", "Dynamic_Price_Percentage"]].head(10)

,Station Name,Date,Time Period,Foot_Traffic,Dynamic_Price_Percentage
99641,Smithsonian,2024-03-24,Entries (PM Peak (3pm-7pm)),19275,112.771829
62743,Rosslyn,2024-10-27,Entries (Midday (9:30am-3pm)),19233,184.798636
99647,Smithsonian,2024-03-30,Entries (PM Peak (3pm-7pm)),16869,52.579757
135483,Smithsonian,2024-07-04,Entries (Evening (7pm-12am)),16405,184.798636
129365,Navy Yard-Ballpark,2024-07-19,Entries (Evening (7pm-12am)),16222,174.616687
129323,Navy Yard-Ballpark,2024-06-07,Entries (Evening (7pm-12am)),15788,117.350089
129359,Navy Yard-Ballpark,2024-07-13,Entries (Evening (7pm-12am)),15085,184.798636
129366,Navy Yard-Ballpark,2024-07-20,Entries (Evening (7pm-12am)),14450,148.434238
125692,L'Enfant Plaza,2024-07-04,Entries (Evening (7pm-12am)),13448,184.798636
129349,Navy Yard-Ballpark,2024-07-03,Entries (Evening (7pm-12am)),13359,137.886034


In [88]:
import pandas as pd

# Randomly select 20 rows from the dataset
df_sample = df_sorted.sample(n=20, random_state=42)  # Set random_state for consistency

# Display the table in the same format
df_sample[["Station Name", "Date", "Time Period", "Foot_Traffic", "Dynamic_Price_Percentage"]]


,Station Name,Date,Time Period,Foot_Traffic,Dynamic_Price_Percentage
76211,Capitol South,2024-12-07,Entries (PM Peak (3pm-7pm)),892,-60.783821
15069,Greenbelt,2024-03-17,Entries (AM Peak (Open-9:30am)),268,-79.273009
51276,Greensboro,2024-06-26,Entries (Midday (9:30am-3pm)),206,-0.961538
56394,Metro Center,2024-06-20,Entries (Midday (9:30am-3pm)),3066,15.567282
41007,Clarendon,2024-05-26,Entries (Midday (9:30am-3pm)),908,21.669219
10920,Farragut North,2024-11-07,Entries (AM Peak (Open-9:30am)),669,43.430322
53782,King St-Old Town,2024-05-01,Entries (Midday (9:30am-3pm)),1118,-9.693053
80197,Dulles Airport,2024-10-31,Entries (PM Peak (3pm-7pm)),994,-8.338822
124354,Hyattsville Crossing,2024-11-07,Entries (Evening (7pm-12am)),264,-16.719243
49714,Gallery Place,2024-03-12,Entries (Midday (9:30am-3pm)),2339,1.374528
